In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import loguniform
from matplotlib import pyplot as plt
import seaborn as sns

df = pd.read_csv("Admission_Predict.csv") # Veri setini yüklüyoruz

display(df.head())
display(df.info())
display(df.describe())
print(df.isnull().sum())
print(df.duplicated().sum())

# Veri seti hakkında genel bilgi ediniyoruz

df["Chance of Admit Binary_75"] = (df["Chance of Admit "] >= 0.75).astype(int) # 0.75 üzerinde olanlar 1, altinda olanlar 0


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.3 KB


None

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64
0


In [2]:
y = df["Chance of Admit Binary_75"]
X = df.drop(columns=["Serial No.", "Chance of Admit ","Chance of Admit Binary_75"])
# Bağımlı ve bağımsız değişkenleri ayırıyoruz.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42) # Veri setini eğitim ve test setlerine ayırıyoruz.

scaler = StandardScaler() # StandardScaler kullanıyoruz.

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Verileri standartlaştırıyoruz.

In [3]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
# Modeli eğitim verisi ile eğitiyoruz.

y_pred = model.predict(X_test_scaled) # Test verisi ile tahmin yapıyoruz.

conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)
# Confusion matrix'i yazdırıyoruz.

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluk Oranı: {accuracy:.2f}")
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))
# Sınıflandırma raporunu yazdırıyoruz.


Confusion Matrix:
[[42  5]
 [ 2 31]]
Model Doğruluk Oranı: 0.91

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.95      0.89      0.92        47
           1       0.86      0.94      0.90        33

    accuracy                           0.91        80
   macro avg       0.91      0.92      0.91        80
weighted avg       0.92      0.91      0.91        80



In [4]:
sgd_model = SGDClassifier(loss="log_loss", max_iter=1000, tol=1e-4, random_state=42, verbose=1) # SGDClassifier modelini oluşturuyoruz.

# Modeli SGDClassifier ile oluşturuyoruz.
sgd_model.fit(X_train_scaled, y_train.values.ravel())

-- Epoch 1
Norm: 37.88, NNZs: 7, Bias: -9.923011, T: 320, Avg. loss: 3.826109
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 24.18, NNZs: 7, Bias: 3.722609, T: 640, Avg. loss: 2.875460
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 25.39, NNZs: 7, Bias: -5.767422, T: 960, Avg. loss: 2.197797
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 24.43, NNZs: 7, Bias: -1.318072, T: 1280, Avg. loss: 1.843060
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 18.60, NNZs: 7, Bias: -1.380548, T: 1600, Avg. loss: 1.827404
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 13.74, NNZs: 7, Bias: 0.248766, T: 1920, Avg. loss: 1.477562
Total training time: 0.00 seconds.
-- Epoch 7
Norm: 13.88, NNZs: 7, Bias: 3.439725, T: 2240, Avg. loss: 1.426425
Total training time: 0.00 seconds.
-- Epoch 8
Norm: 8.81, NNZs: 7, Bias: -8.539590, T: 2560, Avg. loss: 1.010588
Total training time: 0.00 seconds.
-- Epoch 9
Norm: 8.95, NNZs: 7, Bias: -0.537057, T: 2880, Avg. loss: 1.128493
Total training ti

SGDClassifier(loss='log_loss', random_state=42, tol=0.0001, verbose=1)

In [5]:
# Veri setinin üzerinden 54 kere geçtikten sonra modelin ağırlıklarını ve bias'ını yazdırıyoruz.
print(f"Son Ağırlıklar (w): {sgd_model.coef_}")
print(f"\nSon Bias (b): {sgd_model.intercept_} \n")


importance_df = pd.DataFrame({"Özellik": X.columns,"Katsayı": model.coef_[0]}) # Önem sırasını yazdırabilmek için Özellik isimleri ve önem katsayılarını içeren bir DataFrame oluşturuyoruz.

# Katsayının mutlak değeriyle önem sıralaması yapıyoruz.
importance_df["Mutlak Katsayı"] = importance_df["Katsayı"].abs()
importance_df = importance_df.sort_values(by="Mutlak Katsayı", ascending=False)

print("Önem sırasına göre değişkenlerin katsayıları:")
print(importance_df[["Özellik", "Katsayı"]])
# Özelliklerin önem sırasını yazdırıyoruz.

Son Ağırlıklar (w): [[ 1.04216374  0.19986285  0.67351113  0.78222878 -0.29913669  2.20410501
   0.57554193]]

Son Bias (b): [-1.13652884] 

Önem sırasına göre değişkenlerin katsayıları:
             Özellik   Katsayı
5               CGPA  1.760462
0          GRE Score  0.718233
3                SOP  0.425557
6           Research  0.411541
1        TOEFL Score  0.346984
2  University Rating  0.309939
4               LOR   0.252717


In [6]:
df["Chance of Admit Binary_70"] = (df["Chance of Admit "] >= 0.70).astype(int) # 0.70 üzerinde olanlar 1, altinda olanlar 0
y = df["Chance of Admit Binary_70"] # Bağımlı değişkeni 0.70 olan ile değiştiriyoruz.
X = df.drop(columns=["Serial No.", "Chance of Admit ", "Chance of Admit Binary_75", "Chance of Admit Binary_70"]) # Bağımsız değişkenleri yeni gelen veri setine göre ayarlıyoruz.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

scaler = StandardScaler()
X_train_scaled_70 = scaler.fit_transform(X_train)
X_test_scaled_70 = scaler.transform(X_test)
# Verileri standartlaştırıyoruz.

model_70 = LogisticRegression(max_iter=1000)
model_70.fit(X_train_scaled_70, y_train)
y_pred_70 = model_70.predict(X_test_scaled_70) 
# Güncellenmiş bağımlı değişken ile modelimizi eğitiyoruz ve test verisi ile tahmin yapıyoruz.

conf_matrix = confusion_matrix(y_test, y_pred_70)
print("\nConfusion Matrix:")
print(conf_matrix)
# Confusion matrix'i yazdırıyoruz.

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Doğruluk Oranı: {accuracy:.2f}")
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))
# Modelin doğruluk oranını ve sınıflandırma raporunu yazdırıyoruz.


Confusion Matrix:
[[28  4]
 [ 5 43]]
Model Doğruluk Oranı: 0.82

Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.70      0.97      0.82        32
           1       0.97      0.73      0.83        48

    accuracy                           0.82        80
   macro avg       0.84      0.85      0.82        80
weighted avg       0.87      0.82      0.83        80



In [7]:
df["Chance of Admit Binary_80"] = (df["Chance of Admit "] >= 0.80).astype(int) # 0.80 üzerinde olanlar 1, altinda olanlar 0
y_80 = df["Chance of Admit Binary_80"] # Bağımlı değişkeni 0.80 olan ile değiştiriyoruz.
X_80 = df.drop(columns=["Serial No.", "Chance of Admit ", "Chance of Admit Binary_75", "Chance of Admit Binary_70", "Chance of Admit Binary_80"])
# Bağımsız değişkenleri yeni gelen veri setine göre ayarlıyoruz.

# yeniden bölümleme yapıyoruz
X_train_80, X_test_80, y_train_80, y_test_80 = train_test_split(X_80, y_80, test_size=0.20, random_state=42)

scaler_80 = StandardScaler()
X_train_scaled_80 = scaler_80.fit_transform(X_train_80)
X_test_scaled_80 = scaler_80.transform(X_test_80)
# Verileri standartlaştırıyoruz.

model_80 = LogisticRegression(max_iter=1000)
model_80.fit(X_train_scaled_80, y_train_80) # Güncellenmiş y_train_80 ile eğitiyoruz.
y_pred_80 = model_80.predict(X_test_scaled_80) # Güncellenmiş X_test_scaled_80 ile tahmin ediyoruz.

print("\nConfusion Matrix (0.80):")
print(confusion_matrix(y_test_80, y_pred_80)) # Güncellenmiş y_test_80 ile karşılaştır
accuracy_80 = accuracy_score(y_test_80, y_pred_80)
print(f"Model Doğruluk Oranı (0.80): {accuracy_80:.2f}")
print("\nSınıflandırma Raporu (0.80):")
print(classification_report(y_test_80, y_pred_80))
# Modelin doğruluk oranını ve sınıflandırma raporunu yazdırıyoruz.


Confusion Matrix (0.80):
[[49  2]
 [ 1 28]]
Model Doğruluk Oranı (0.80): 0.96

Sınıflandırma Raporu (0.80):
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        51
           1       0.93      0.97      0.95        29

    accuracy                           0.96        80
   macro avg       0.96      0.96      0.96        80
weighted avg       0.96      0.96      0.96        80



Modellerimize baktığımızda genel olarak iyi sonuçlar aldığımızı görebiliriz. En iyi sonuç 0.80 eşik değeri ile %96 accuracy puanı ile elde edilmiştir. Bu sebeple Logistic regression için tercih edilen eşik değeri 0.80'dir.

Modelin geliştirilmesi için farklı yöntemler deneyebiliriz. Örneğin, modelin hiperparametrelerini en iyi halinde ayarlamak için GridSearchCV veya RandomizedSearchCV gibi yöntemler kullanabiliriz. 

Ayrıca, modelin performansını artırmak için daha karmaşık modeller deneyebiliriz. 
Örneğin, karar ağaçları (Decision Tree), rastgele ormanlar (Random Forest) veya destek vektör makineleri (Support Vector Machine (SVM)) gibi daha karmaşık algoritmalar kullanılabilir.
Bu tarz modelleri kullandıktan sonra Ensemble öğrenme metodu ile hepsinin karışımını kullanarak daha sağlıklı tahminler yapılabilir.

Bunun yanı sıra, modelin performansını artırmak için daha fazla veri toplamak veya veri ön işleme adımlarını iyileştirmek de faydalı olabilir. 
Özellikle özellik mühendisliği yapmak gibi adımlar modelin başarısını artırabilir. 

Tek bir eğitim‑test bölmesi yerine, k‑fold cross‑validation uygulayarak modelin farklı veri dilimlerinde ne kadar istikrarlı performans gösterdiğini ölçebilir, aşırı öğrenmeyi (overfitting) tespit edebiliriz.

HİPERPARAMETRE OPTİMİZASYONU

In [ ]:
# Hiperparametre aralıklarını belirliyoruz
param_grid = [
    {
        'solver': ['saga'],
        'penalty': ['elasticnet'],
        'C': [5, 10, 15, 20],
        'l1_ratio': [0.6, 0.65, 0.7, 0.75, 0.8],
        'max_iter': [2500, 3000, 3500, 4000]
    },
    {
        'solver': ['lbfgs'],
        'penalty': ['l2'],
        'C': [0.01, 0.1, 1, 5, 10, 15, 20],
        'max_iter': [100, 200, 500, 1000]
    },
    {
        'solver': ['liblinear'],
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 5, 10, 15, 20],
        'max_iter': [100, 200, 500, 1000]
    }
]
# GridSearchCV ile hiperparametre optimizasyonu
grid_search = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train_scaled_80, y_train_80)
grid = grid_search.best_estimator_
y_pred_gs = grid.predict(X_test_scaled_80)

print("GridSearchCV En İyi Parametreler:", grid_search.best_params_)
print("GridSearchCV En İyi Skor:", accuracy_score(y_test_80, y_pred_gs))
print("\n")

# Random Search için hiperparametre aralıklarını belirliyoruz
param_dist_rs = {
    'solver': ['saga'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': loguniform(1e-3, 1e2),
    'l1_ratio': np.random.uniform(0, 1, 100),
    'max_iter': [1000, 2000, 3000, 4000]
}

# RandomizedSearchCV ile hiperparametre optimizasyonu
random_search = RandomizedSearchCV(LogisticRegression(random_state=42), param_distributions=param_dist_rs, cv=5, scoring='accuracy', n_jobs=-1, n_iter=20, random_state=42, verbose=1)
random_search.fit(X_train_scaled_80, y_train_80)
random = random_search.best_estimator_
y_pred_rs = random.predict(X_test_scaled_80)

print("RandomizedSearchCV En İyi Parametreler:", random_search.best_params_)
print("RandomizedSearchCV En İyi Skor:", accuracy_score(y_test_80, y_pred_rs))


Fitting 5 folds for each of 164 candidates, totalling 820 fits
GridSearchCV En İyi Parametreler: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
GridSearchCV En İyi Skor: 0.975


Fitting 5 folds for each of 20 candidates, totalling 100 fits
RandomizedSearchCV En İyi Parametreler: {'C': 0.14445251022763064, 'l1_ratio': 0.10238902694673535, 'max_iter': 3000, 'penalty': 'elasticnet', 'solver': 'saga'}
RandomizedSearchCV En İyi Skor: 0.95


In [ ]:
# Grid Search Confusion Matrix
print("\nGridSearchCV Confusion Matrix:")
cm_gs = confusion_matrix(y_test_80, y_pred_gs)
print(cm_gs)

# Grid Search Sınıflandırma Raporu
print("\nGridSearchCV Sınıflandırma Raporu:")
report_gs = classification_report(y_test_80, y_pred_gs)
print(report_gs)

# Random Search Confusion Matrix
print("\n\nRandomizedSearchCV Confusion Matrix:")
cm_rs = confusion_matrix(y_test_80, y_pred_rs)
print(cm_rs)

# Random Search Sınıflandırma Raporu
print("\nRandomizedSearchCV Sınıflandırma Raporu:")
report_rs = classification_report(y_test_80, y_pred_rs)
print(report_rs)


GridSearchCV Confusion Matrix:
[[49  2]
 [ 0 29]]

GridSearchCV Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        51
           1       0.94      1.00      0.97        29

    accuracy                           0.97        80
   macro avg       0.97      0.98      0.97        80
weighted avg       0.98      0.97      0.98        80



RandomizedSearchCV Confusion Matrix:
[[49  2]
 [ 2 27]]

RandomizedSearchCV Sınıflandırma Raporu:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        51
           1       0.93      0.93      0.93        29

    accuracy                           0.95        80
   macro avg       0.95      0.95      0.95        80
weighted avg       0.95      0.95      0.95        80



1. Hiperparametre optimizasyonu modelin başarısını artırdı mı?

Modelimiz halihazırda çok başarılıdır. Üzerine yazdığımız GridSearch modelin performansını bir miktar arttırmıştır.
RandomizedSearchCV ile yapılan denemelerde ise modelin performansı genel olarak düşse de, her çalıştırdığımızda farklı sonuçlar geldiğinden dolayı bazı denemelerde 0.96 değerlerini de vermiştir.
Bu sonuçlardan dolayı GridSearchCV ile hiperparametre optimizasyonu, modelin performansını arttırmıştır.

2. Sınıf 0 ve Sınıf 1 arasında dengesizlik var mı?

Sınıf 0, 51 support değerine sahip olmasına rağmen, Sınıf 1, 29 support değerine sahiptir. Sınıf 1 azınlık olduğundan dolayı dengeli değildir.

- Precision ve Recall değerleri dengeli mi?

Precision için 1 ve 0.94, Recall için 0.96 ve 1, F1 skoru için ise 0.98 ve 0.97 başarı oranları gözlemledik. Farklılıklar olsa da bu farklılıklar çok düşük olduğundan dolayı modeli performans açısından hem yüksek hem de dengeli kabul edebiliriz.

3. Modeli iyileştirmek için başka yöntemler gerekli mi?

Temel modelin zaten "çok başarılı" olduğu ve hiperparametre optimizasyonunun bu başarıyı fazla ileriye taşımadığını gözlemledik. GridSearchCV ile elde edilen %97.5 doğruluk da modelin iyi bir performans sergilediğini doğrulamaktadır. Eğer bu performans seviyesi kabul edilebilir düzeydeyse, ek yöntemler uygulamak zorunlu olmayabilir. Ancak, projenin hassasiyetine göre daha da iyileştirme hedefleniyorsa hata analizi gibi adımlar düşünülebilir.
